# Joint Embedding of Protein Signals and Amino Acid Sequences

This notebook aims to create joint embeddings of protein signals and their corresponding amino acid sequences in the same vector space. 

TODO:
- Dataset
    - Make the nanopore readings a consistent size so an input dem can be 
- Model
    - Save model weights on completition
- Evaluation
    - Crete a dummy classifier (not sure how)
    - Compare to the CNN and RF models
    - Try to do a 1:1 comparison between existing models and my embedding model

## Configuration

In [1]:
nanopore_readings_path = './data/run_df.json'
EMBEDDING_DIM = 64
HIDDEN_DIM = 128
BATCH_SIZE = 32
EPOCHS = 20

## Imports

In [2]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os, time
# from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
# import torch.nn.functional as F
import torch.optim as optim
import json
# import math
# import warnings
# import scipy.signal as scisignal
# import sklearn.utils.class_weight as class_weight
# from sklearn.decomposition import PCA

## Dataset

In [3]:
"""
A - Alanine
R - Arginine
N - Asparagine
D - Aspartic Acid
C - Cysteine
E - Glutamic Acid
Q - Glutamine
G - Glycine
H - Histidine
I - Isoleucine
L - Leucine
K - Lysine
M - Methionine
F - Phenylalanine
P - Proline
S - Serine
T - Threonine
W - Tryptophan
Y - Tyrosine
V - Valine
"""
# I would prefer these to be sorted, but I am maintaining the order from the original code.
amino_acids_indexes = {aa:i for i, aa in enumerate('CSAGTVNQMILYWFPHRKDE')}

run_to_peptide = {
    '20220824_run02_a': 'HDKER',
    '20220826_run01_a': 'GNQST',
    '20220826_run02_a': 'FYWCP',
    '20220826_run03_a': 'AVLIM',
    '20220907_run01_a': 'GNQST',
    '20221010_run02_a': 'GNQST',
    '20221011_run02_a': 'HDKER',
    '20221026_run01_a': 'VGDNY',
    '20221028_run01_a': 'TWAFH',
    '20221028_run02_a': 'PRMQE',
    '20221107_run01_a': 'TWAFH',
    '20221108_run01_a': 'TWAFH',
    '20221109_run01_a': 'VGDNY',
    '20221109_run02_a': 'PRMQE',
    '20221109_run03_a': 'KSILC',
    '20221109_run04_a': 'FYWCP',
    '20221110_run02_a': 'AVLIM',
    '20221121_run01_a': 'KSILC',
    '20221122_run01_a': 'KSILC',
    '20221122_run02_a': 'PRMQE',
    '20221122_run03_a': 'KSILC', 
    '20221213_run02_a': 'FYWCP',
    '20221214_run01_a': 'FYWCP',
    '20221214_run04_a': 'FYWCP',
    '20221219_run01_a': 'VGDNY',
    '20221220_run01_a': 'FYWCP',
    '20221220_run04_a': 'AVLIM',
    '20221221_run01_a': 'TWAFH',
    '20221221_run02_a': 'KSILC',
 }

vocab_size = len(amino_acids_indexes)

def path_to_dataframe(nanopore_readings_path):
    """
    Converts the nanopore readings JSON file to a pandas DataFrame with readings as numpy arrays.
    """
    with open(nanopore_readings_path, 'r') as f:
        loaded_json = json.load(f)
    data_dict = loaded_json['data']
    records = [
        {'run_id': run_id, 
         'readings': np.array(list(readings_dict.values())[0], dtype=np.float32), 
         'peptide': run_to_peptide[run_id]}
        for run_id, readings_dict in data_dict.items()
    ]
    return pd.DataFrame(records)

nanopore_df = path_to_dataframe(nanopore_readings_path)

class PastorDataset(Dataset):
    """
    Custom dataset nanopore readings and amino acid sequences.
    """
    def __init__(self, nanopore_df, amino_acid_tokenizer):
        self.nanopore_df = nanopore_df
        self.amino_acid_tokenizer = amino_acid_tokenizer

    def __len__(self):
        return len(self.nanopore_df)

    """
    Need to revisit this
    """
    def __getitem__(self, idx): 
        nanopore_sample = torch.tensor(self.nanopore_df.iloc[idx].values, dtype=torch.float32)

        # Positive amino acid sample
        positive_amino_acid = self.nanopore_df[idx].peptide
        positive_amino_acid_tokenized = self.amino_acid_tokenizer(positive_amino_acid)

        # Negative amino acid sample
        negative_idx = np.random.randint(0, len(self.nanopore_df))
        while negative_idx == idx or self.nanopore_df.iloc[negative_idx].peptide == positive_amino_acid:
            negative_idx = np.random.randint(0, len(self.amino_acids))
        negative_amino_acid = self.nanopore_df[negative_idx]
        negative_amino_acid_tokenized = self.amino_acid_tokenizer(negative_amino_acid)

        return (
            nanopore_sample,
            positive_amino_acid_tokenized,
            negative_amino_acid_tokenized
        )

# def amino_acid_tokenizer(sequence, char_to_int, max_length=100):
#     """
#     Tokenizes an amino acid sequence.
#     """
#     encoded = [char_to_int[char] for char in sequence]
#     padded = np.zeros(max_length, dtype=np.int64)
#     length = len(encoded)
#     if length > max_length:
#         length = max_length
#     padded[:length] = encoded[:length]
#     return torch.tensor(padded)

dataset = PastorDataset(nanopore_df, lambda x: torch.tensor([amino_acids_indexes[aa] for aa in x], dtype=torch.long))
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

## Model Setup

In [4]:
class NanoporeReadingEncoder(nn.Module):
    """
    Encoder for the run_df data. It's a simple Multi-Layer Perceptron (MLP).
    """
    def __init__(self, input_dim, embedding_dim):
        super(NanoporeReadingEncoder, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, embedding_dim)
        )

    def forward(self, x):
        return self.model(x)

class PastorEncoder(nn.Module):
    """
    Encoder for the PASTOR amino acid sequences. It uses a Bidirectional LSTM.
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(PastorEncoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, embedding_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, (hidden, _) = self.lstm(embedded)
        # We use the concatenation of the final forward and backward hidden states
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        return self.fc(hidden)

class JointEmbeddingModel(nn.Module):
    """
    The main model that combines the two encoders.
    """
    def __init__(self, run_df_encoder, amino_acid_encoder):
        super(JointEmbeddingModel, self).__init__()
        self.run_df_encoder = run_df_encoder
        self.amino_acid_encoder = amino_acid_encoder

    def forward(self, run_df, amino_acid):
        run_df_embedding = self.run_df_encoder(run_df)
        amino_acid_embedding = self.amino_acid_encoder(amino_acid)
        return run_df_embedding, amino_acid_embedding

In [5]:
class ContrastiveLoss(nn.Module):
    """
    Contrastive loss function.
    """
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = nn.functional.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

def train_model(model, data_loader, epochs=10, learning_rate=0.001):
    """
    Training loop for the joint embedding model.
    """
    criterion = ContrastiveLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(epochs):
        for i, (run_df_sample, pos_amino_acid, neg_amino_acid) in enumerate(data_loader):
            run_df_sample = run_df_sample.to(device)
            pos_amino_acid = pos_amino_acid.to(device)
            neg_amino_acid = neg_amino_acid.to(device)

            # Positive pair
            optimizer.zero_grad()
            run_df_embedding, pos_amino_acid_embedding = model(run_df_sample, pos_amino_acid)
            loss_pos = criterion(run_df_embedding, pos_amino_acid_embedding, torch.zeros(run_df_sample.size(0), 1).to(device))

            # Negative pair
            run_df_embedding, neg_amino_acid_embedding = model(run_df_sample, neg_amino_acid)
            loss_neg = criterion(run_df_embedding, neg_amino_acid_embedding, torch.ones(run_df_sample.size(0), 1).to(device))

            loss = loss_pos + loss_neg
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

    return model

In [6]:
def evaluate_model(model, data_loader, top_k=5):
    """
    Evaluates the model by checking the retrieval accuracy.
    """
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    correct = 0
    total = 0

    with torch.no_grad():
        for run_df_sample, pos_amino_acid, _ in data_loader:
            run_df_sample = run_df_sample.to(device)
            pos_amino_acid = pos_amino_acid.to(device)

            run_df_embedding, pos_amino_acid_embedding = model(run_df_sample, pos_amino_acid)

            # Create a batch of negative samples for evaluation
            # (In a real scenario, you'd use a larger set of candidates)
            batch_size = run_df_sample.size(0)
            candidate_indices = np.random.choice(len(data_loader.dataset), size=batch_size, replace=False)
            candidate_amino_acids = [data_loader.dataset.amino_acids[i] for i in candidate_indices]
            candidate_amino_acids_tokenized = torch.stack(
                [data_loader.dataset.amino_acid_tokenizer(seq) for seq in candidate_amino_acids]
            ).to(device)
            _, candidate_embeddings = model(run_df_sample, candidate_amino_acids_tokenized)

            # Calculate distances
            distances = torch.cdist(run_df_embedding, torch.cat([pos_amino_acid_embedding, candidate_embeddings]))

            # Check if the positive sample is in the top_k closest
            _, top_k_indices = torch.topk(distances, top_k, largest=False)
            correct += (top_k_indices == 0).any(dim=1).sum().item()
            total += batch_size

    print(f"Top-{top_k} Accuracy: {100 * correct / total}%")

## Running the model

In [ ]:
# --- 4. Model Initialization ---
run_df_encoder = NanoporeReadingEncoder(input_dim=nanopore_df, embedding_dim=EMBEDDING_DIM)
amino_acid_encoder = PastorEncoder(vocab_size=vocab_size, embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM)
model = JointEmbeddingModel(run_df_encoder, amino_acid_encoder)


# --- 5. Training ---
print("--- Training ---")
trained_model = train_model(model, data_loader, epochs=EPOCHS)


# --- 6. Evaluation ---
print("\n--- Evaluation ---")
evaluate_model(trained_model, data_loader)

NameError: name 'run_df' is not defined

In [8]:
ex = path_to_dataframe(nanopore_readings_path)

In [ ]:
len(ex.readings[1])

14023

In [ ]:
# Need to normalize the length of the readings for a consistent input size